Create a program that perform below mentioned task upon recognizing a particular face. 

📌 When it recognize your face then - 
👉 It send mail to your mail id by writing this is face of your_name. 
👉 Second it send whatsapp message to your friend, it can be anything. 

📌 When it recognize second  face, it can be your friend or family members face.
👉 Create EC2 instance in the AWS using CLI. 
👉 Create 5 GB EBS volume and attach it to the instance. 

In [1]:
import cv2
import numpy as np
model=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [2]:
#to send email
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
def send_email(user, recipient):
    mail_content = """
    This is the face of Natasha.
    Your face has been detected!!!
    You have logged into your laptop.
    """
    #The mail addresses and password
    sender_address = user
    sender_pass = '26260169'
    receiver_address = recipient
    #Setup the MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = 'Log-in detected.'   #The subject line
    #The body and the attachments for the mail
    message.attach(MIMEText(mail_content, 'plain'))
    #Create SMTP session for sending the mail
    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_address, sender_pass) #login with mail_id and password
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()
    print('Mail Sent')

In [ ]:
cap=cv2.VideoCapture(0)

In [3]:
#to send a whatsapp message
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
def whatsapp():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    message='Meha! Tasha has logged in.'
    driver.get("https://web.whatsapp.com/") 
    sleep(20)
    driver.find_element_by_xpath('//span[@title="Megaa"][@dir="auto"]').click()

    driver.find_element_by_xpath('//div[@dir="ltr"][@data-tab="6"][@spellcheck="true"]').send_keys(message, Keys.ENTER)
    print('WhatsApp message has been sent')

In [ ]:
#to collect face data
cap=cv2.VideoCapture(0)
count=0



while True:
    ret,photo= cap.read()

    face=model.detectMultiScale(photo)
    if len(face)==0:
        pass
    else:
        x1=face[0][0]
        y1=face[0][1]
        x2= x1+face[0][2]
        y2= y1+face[0][3]

        rectphoto=cv2.rectangle(photo,(x1,y1),(x2,y2),[0,255,0],5)
        cv2.imshow('camera is live...',rectphoto)
        
        cropped_photo=photo[y1:y2,x1:x2]
        cropped_photo=cv2.resize(cropped_photo, (200,200))
        cropped_photo=cv2.cvtColor(cropped_photo, cv2.COLOR_BGR2GRAY)
        cropped_photo_path="./tashaface/"+str(count)+ '.jpg'
        count+=1
        cv2.imwrite(cropped_photo_path,cropped_photo)
        
        if cv2.waitKey(100)==13 or count==100:
            cap.release()
            break
        intruder=True
            
        
cv2.destroyAllWindows()

In [4]:
#train the model to correctly detect your face
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './tashaface/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

tash_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
tash_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


In [6]:
#to detect and check face

import time
import os
cap=cv2.VideoCapture(0)
detect_tasha=False
detect_other=False

while True:
    ret,photo= cap.read()

    face=model.detectMultiScale(photo)
    if len(face)==0:
        pass
    else:
        x1=face[0][0]
        y1=face[0][1]
        x2= x1+face[0][2]
        y2= y1+face[0][3]

        aphoto=cv2.rectangle(photo,(x1,y1),(x2,y2),[0,255,0],5)
        cv2.imshow('camera is live...',aphoto)
        
        cropped_photo=photo[y1:y2,x1:x2]
        cropped_photo=cv2.resize(cropped_photo, (200,200))
        cropped_photo=cv2.cvtColor(cropped_photo, cv2.COLOR_BGR2GRAY)
        
        detection_result=tash_model.predict(cropped_photo)
        
        
        if detection_result[1]<500:
            confidence = int( 100 * (1 - (detection_result[1])/400) )
            #display_string = str(confidence) + '% Confident it is tasha'
            
        if confidence>85:
            detect_tasha=True
            cv2.putText(photo, "This is Natasha Vergis", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('face',photo)
            
        else:
            cv2.putText(photo, "This is NOT Natasha Vergis.Will create aws instance and attach a volume for you.", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            detect_other=True
            cv2.imshow('face',photo)
        
        if cv2.waitKey(100)==13:
            cap.release()
            break
        
            
        
cv2.destroyAllWindows()

if detect_tasha==True:
    cap.release()
    whatsapp()
    send_email("tashdumdum@gmail.com", "tashavergiskr@gmail.com")
    
if detect_other==True:
    cap.release()
    cv2.destroyAllWindows()
    os.system("aws ec2 run-instances  --image-id ami-010aff33ed5991201 --instance-type t2.micro  --subnet-id subnet-ecf11a87  --count 1 --security-group-ids sg-0067e1de928834ac7   --key-name aws21")
    print("Instance Launched")
    print("Please wait for 2 minutes instance is initializing")
    os.system("aws ec2 create-volume --availability-zone ap-south-1a --size 5 --volume-type gp2 --tag-specification  ResourceType=volume,Tags=[{Key=face,Value=volume}]")
    print("Volume Created")
    i=input("Enter instance id : ")
    j=input("Enter volume id :")
    time.sleep(30)
    os.system("aws ec2 attach-volume --instance-id {} --volume-id {} --device /dev/sdk".format(i,j))
    print("Volume Successfully attached to the instance")
    

Instance Launched
Please wait for 2 minutes instance is initializing
Volume Created
Enter instance id : i-07c2a2f528473caf4
Enter volume id :vol-0d05ec3c5272ca1b9
Volume Successfully attached to the instance
